In [1]:
import duckdb

In [ ]:
# con.close()

In [2]:
con = duckdb.connect('phish_pipeline.duckdb')
con.sql('select * from information_schema.tables')


┌────────────────┬──────────────┬───────────────────────┬────────────┬──────────────────────────────┬──────────────────────┬───────────────────────────┬──────────────────────────┬────────────────────────┬────────────────────┬──────────┬───────────────┬───────────────┐
│ table_catalog  │ table_schema │      table_name       │ table_type │ self_referencing_column_name │ reference_generation │ user_defined_type_catalog │ user_defined_type_schema │ user_defined_type_name │ is_insertable_into │ is_typed │ commit_action │ TABLE_COMMENT │
│    varchar     │   varchar    │        varchar        │  varchar   │           varchar            │       varchar        │          varchar          │         varchar          │        varchar         │      varchar       │ varchar  │    varchar    │    varchar    │
├────────────────┼──────────────┼───────────────────────┼────────────┼──────────────────────────────┼──────────────────────┼───────────────────────────┼──────────────────────────┼──────────────

In [86]:
user_attendance__data = con.sql('select * from phish_data.user_attendance__data')
user_attendance  = con.sql('select * from phish_data.user_attendance')
att = user_attendance.select('uid', 'showid')
setlists  = con.sql('select * from phish_data.setlists')
setlists
att
user_attendance

┌─────────┬────────────────┬────────────┬──────────┬───────────┬─────────┬────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────┬─────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [87]:
con.sql("select * from phish_data.user_attendance__data att where showid = '1326251770'")

┌─────────┬──────────┬────────────┬──────────┬───────────┬─────────┬────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────┬─────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────┬───────────┬─────────┬─────────┬──────────┬────────────────┬─────────┬────────────────────┬────────────────┬───────────────┬────────────────┐
│   uid   │ username │   showid   │ showyear │ showmonth │ showday │  showdate  │                                            permalink                                             │ exclude_from_stats 

In [88]:
sql = '''
select *
from
phish_data.user_attendance__data att
full join phish_data.setlists setlists on trim(att.showid) = trim(setlists.showid)
'''

con.sql(sql).to_view('user_setlists')

sql = '''
select count(*) from user_setlists

'''
sql = '''
select count(*) from phish_data.user_attendance__data att
'''

con.sql(sql)




┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      2537230 │
└──────────────┘

In [63]:
con.sql('select count(*) from user_setlists')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│            0 │
└──────────────┘

In [18]:
con.close()